In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

In [5]:
# 라이브러리 불러오기
import os
import json
import shutil
from PIL import Image
import re
import time
import pickle
import tempfile
from ultralytics import YOLO

## 예측 코드

In [21]:
# 1. 학습된 best.pt 모델 로드
model = YOLO('runs/detect/train36/weights/best.pt')

# 2. 예측하고자 하는 이미지 파일 경로
# image_path = '[BLUE]00309A_082918_012.jpg'
image_path = 'falsecar.jpg'

# 이미지 예측
results = model.predict(source=image_path , save=True, imgsz=640)  # your_image.jpg 경로에 맞게 수정

# 결과 확인
for result in results:
    result.plot()  # 예측 결과 이미지 출력


image 1/1 /home/j-j11a303/falsecar.jpg: 416x640 1 lane_blue, 33.3ms
Speed: 2.1ms preprocess, 33.3ms inference, 3.9ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict40


In [22]:
# 4. 차량과 차선 객체 분류
vehicle_boxes = []
lane_boxes = []

for box in results[0].boxes:  # results[0]을 사용하여 첫 번째 결과에 접근
    cls = int(box.cls)  # 클래스 인덱스 가져오기
    class_name = results[0].names[cls]  # 클래스 이름 가져오기
    
    # 클래스 이름에 따라 차량과 차선을 구분
    if class_name in ['vehicle_car', 'vehicle_bus', 'vehicle_truck', 'vehicle_bike']:
        vehicle_boxes.append(box)
    elif class_name in ['lane_white', 'lane_blue', 'lane_yellow', 'lane_shoulder']:
        lane_boxes.append(box)

# 5. 차량과 차선 간의 관계 분석
def is_overlapping(box1, box2):
    # 각 경계 상자의 좌표 (x1, y1, x2, y2)
    x1_1, y1_1, x2_1, y2_1 = box1.xyxy[0].tolist()  # 차량 경계 상자 좌표
    x1_2, y1_2, x2_2, y2_2 = box2.xyxy[0].tolist()  # 차선 경계 상자 좌표
    
    # 겹침 여부 확인 (두 상자가 겹치는 경우 True 반환)
    return not (x2_1 < x1_2 or x2_2 < x1_1 or y2_1 < y1_2 or y2_2 < y1_1)


print(vehicle_boxes)

# 6. 불법 주차 여부 판단
for vehicle_box in vehicle_boxes:
    vehicle_coords = vehicle_box.xyxy  # 차량의 경계 상자 좌표
    
    for lane_box in lane_boxes:
        lane_coords = lane_box.xyxy  # 차선의 경계 상자 좌표
        
        # 차량이 차선 위에 있는지 확인 (경계 상자 간의 겹침 여부 계산)
        if is_overlapping(vehicle_box, lane_box):
            lane_type = results[0].names[int(lane_box.cls)]  # 차선의 종류
            print(f"차량이 {lane_type} 차선 위에 있습니다.")
            
            # 파란색 차선 위에 있으면 불법 주차로 판단
            if lane_type in  ['lane_blue', 'lane_yellow', 'lane_shoulder']:
                print("불법 주차입니다.")
            else:
                print("정상 주차입니다.")

[]


In [23]:
print(results)

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'vehicle_car', 1: 'vehicle_bus', 2: 'vehicle_truck', 3: 'vehicle_bike', 4: 'lane_white', 5: 'lane_blue', 6: 'lane_yellow', 7: 'lane_shoulder'}
obb: None
orig_img: array([[[ 95, 130, 120],
        [ 92, 127, 117],
        [ 79, 116, 108],
        ...,
        [ 18,  27,  41],
        [ 14,  21,  36],
        [ 42,  45,  59]],

       [[ 99, 133, 126],
        [ 86, 123, 115],
        [ 96, 133, 125],
        ...,
        [ 18,  27,  41],
        [ 16,  23,  38],
        [ 32,  35,  49]],

       [[ 70, 106, 100],
        [ 98, 134, 128],
        [ 96, 132, 126],
        ...,
        [ 31,  40,  53],
        [ 31,  39,  52],
        [ 31,  34,  48]],

       ...,

       [[155, 148, 151],
        [162, 154, 155],
        [172, 159, 157],
        ...,
        [126, 147, 168],
        [122, 143, 164],
        [126, 147, 168]],

       [[151, 150